# Top K Regularization
* Let only the top activations in a location through.
Must decide:
1. How many of these layers should be in a net? Many or just a few? If idea is to try and manke it like an object detector, should be a few conv layers than a topk
2. What k to use?
3. Does it make sense to factor it into something like 4 blocks where each has a top 1 instead of doing an overall top 4?

In [67]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Nicer way to import the module?
sys.path.append(str(Path.cwd().parent))

from models.cifar import PreActResNet
from utils.display import load_image, show_img
from utils.loading import load_net_cifar

import torch
from torch import nn
from torch.autograd import Variable
from torch.autograd.gradcheck import zero_gradients
import torchvision.transforms as T
import foolbox
import json
from PIL import Image

device = torch.device("cuda:0")
print(torch.cuda.device_count())
print(torch.cuda.current_device())

2
0


## Test top k layer
scatter just wants an array of the same shape as the missing values, so must feed it the top k array

In [64]:
x = torch.randn(1, 2, 3, 3)
zeros = torch.zeros(x.shape)
print(x)
topk_num, topk_idx = torch.topk(x, k=1, dim=1)
print(topk_idx)
zeros.scatter_(dim=1, index=topk_idx, src=topk_num) 
print(zeros)

tensor([[[[-1.3091, -0.2615, -0.8970],
          [-1.3588,  0.0922,  0.5862],
          [-0.8907, -0.9733,  1.6597]],

         [[-0.0337, -1.0471,  1.0161],
          [ 0.5645,  1.6145, -0.7098],
          [ 0.3794, -0.6410,  1.3275]]]])
tensor([[[[1, 0, 1],
          [1, 1, 0],
          [1, 1, 0]]]])
tensor([[[[ 0.0000, -0.2615,  0.0000],
          [ 0.0000,  0.0000,  0.5862],
          [ 0.0000,  0.0000,  1.6597]],

         [[-0.0337,  0.0000,  1.0161],
          [ 0.5645,  1.6145,  0.0000],
          [ 0.3794, -0.6410,  0.0000]]]])


In [81]:
x = torch.randn(1, 4, 3, 3)
x_1 = x
groups = 2
group_size = int(x.shape[1]/2)

for group in range(groups):
    curr_group = x[:, group*group_size: (group+1)*group_size, :, :]
    x[:, group*group_size: (group+1)*group_size, :, :] = curr_group

print(x==x_1)

tensor([[[[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]],

         [[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]],

         [[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]],

         [[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]]]], dtype=torch.uint8)
